In [12]:
import pymysql as pms
import os
import openpyxl
import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.header import Header
from email import encoders
from email.utils import parseaddr
from email.utils import formataddr
import smtplib
import pandas as pd

In [13]:
def get_datas(sql):
    # 一个传入sql导出数据的函数
    # 跟数据库建立连接
    conn = pms.connect(host='sh-cdb-kg6rp4w2.sql.tencentcdb.com', user='Cdata',
                       passwd='Lpsy88334558', database='edu-m', port=61181)
    # 使用 cursor() 方法创建一个游标对象 cursor
    cur = conn.cursor()
    # 使用 execute() 方法执行 SQL
    cur.execute(sql)
    # 获取所需要的数据
    datas = cur.fetchall()
    #关闭连接
    cur.close()
    #返回所需的数据
    return datas

In [3]:
def get_fields(sql):
    # 一个传入sql导出字段的函数
    conn = pms.connect(host='sh-cdb-kg6rp4w2.sql.tencentcdb.com', user='Cdata',
                       passwd='Lpsy88334558', database='edu-m', port=61181)
    cur = conn.cursor()
    cur.execute(sql)
    # 获取所需要的字段名称
    fields = cur.description
    cur.close()
    return fields

In [4]:
def get_excel(data, field, file):
    # 将数据和字段名写入excel的函数
    #新建一个工作薄对象
    new = openpyxl.Workbook()
    #激活一个新的sheet
    sheet = new.active
    #给sheet命名
    sheet.title = '数据展示'
    #将字段名称循环写入excel第一行，因为字段格式列表里包含列表，每个列表的第一元素才是字段名称
    for col in range(len(field)):
        #row代表行数，column代表列数，value代表单元格输入的值，行数和列数都是从1开始，这点于python不同要注意
        _ = sheet.cell(row=1, column=col+1, value=u'%s' % field[col][0])
     #将数据循环写入excel的每个单元格中   
    for row in range(len(data)):
        for col in range(len(field)):
            #因为第一行写了字段名称，所以要从第二行开始写入
            _ = sheet.cell(row=row+2, column=col + 1, value=u'%s' % data[row][col])
            #将生成的excel保存，这步是必不可少的
    newworkbook = new.save(file)
    #返回生成的excel
    return newworkbook

In [5]:
def getYesterday():
    # 获取昨天日期的字符串格式的函数
    #获取今天的日期
    today = datetime.date.today()
    #获取一天的日期格式数据
    oneday = datetime.timedelta(days=1)
    #昨天等于今天减去一天
    yesterday = today - oneday
    #获取昨天日期的格式化字符串
    yesterdaystr = yesterday.strftime('%Y-%m-%d')
    #返回昨天的字符串
    return yesterdaystr

In [6]:
def get_file(sql,name,path):
    print(datetime.datetime.now())
    # 生成数据
    my_data = get_datas(sql)
    # 生成字段名称
    my_field = get_fields(sql)
    # 得到昨天的日期
    yesterdaystr = getYesterday()
    # 文件名称，附件名称
    my_file_name = name + yesterdaystr + '.xlsx'
    # 文件路径，附件路径
    file_path = path+"/"+my_file_name
    print(file_path)
    # 生成excel
    get_excel(my_data, my_field, file_path)
    return file_path,my_file_name



In [7]:
def create_email(email_from, email_to, email_Subject, email_text, path,password,sender,receiver,c_receiver):
    # 输入发件人昵称、收件人昵称、主题，正文，附件地址,附件名称生成一封邮件
    
    # 邮件设置
    msg = MIMEMultipart()
    msg["Subject"] = email_Subject
    msg["to"] = email_to
    msg["Cc"] = c_receiver
    msg["from"] = email_from
    # 邮件正文
    body = email_text
    # 添加邮件正文
    msg.attach(MIMEText(body,'plain','utf-8'))
    
    #读取附件的内容
    file_names = os.listdir(path)
    for file in file_names:
        file_path = path+"/"+file
        xlsxpart = MIMEApplication(open(file_path,'rb').read())
        xlsxpart.add_header('Content-Disposition','attachment',filename=file)
        msg.attach(xlsxpart)
    try:     
        # 设置邮箱服务器地址及端口email_client = smtplib.SMTP_SSL("smtp.exmail.qq.com", 465)
        smtp_server = "smtp.exmail.qq.com"
        server = smtplib.SMTP_SSL(smtp_server,465)
        server.set_debuglevel(1)
        # 登录邮箱
        server.login(sender,password)
        # 发送邮件
        server.sendmail(sender,receiver.split(",")+c_receiver.split(","),msg.as_string())
        # 断开服务器连接
        server.quit()
        print("邮件发送成功")
    except:
        print("邮件发送失败")

In [8]:
def send(path) :
    email_from = '数据组自动报表机器人'
    email_to = '在线教育运营部'
    yesterdaystr = getYesterday()
    # 邮件标题
    email_Subject = '在线教育销售相关数据' + yesterdaystr
    # 邮件正文
    email_text = '各位堂兄堂妹,\n\t附件为每日数据，请查收！\n\n数据团队'
    sender = 'loujunjie@lpt.onexmail.com'
    password = "XC8oU2Mp28kf6286"
    receiver =['zhangyuexia@liupinedu.com','zhangxianjiao@lpt.onexmail.com','huangpeng@liupinedu.com']#接收人邮箱列表
    c_receiver = 'zhangyuexia@liupinedu.com' # 抄送人
    # 发送邮件
    create_email(email_from, email_to, email_Subject, email_text, path,password,sender,receiver,c_receiver)
    print(datetime.datetime.now())

In [9]:
sql1 = """select 
  a.add_date as 统计日期,
  a.成人硬笔展示量,
  a.软笔展示量,
  a.少儿展示量,
  a.总展示量,
  b.`课程包订单数(订单金额>1)`,
  b.`课程包金额(订单金额>1)`,
  c.`静默课程数(订单金额>1)`,
  c.`静默金额(订单金额>1)`
from(
select 
  from_unixtime(esml.addtime,'%Y-%m-%d') as add_date,
  count(case when esc.id = 1 then 1 else null end) as  软笔展示量,
  count(case when esc.id = 2 then 1 else null end) as  成人硬笔展示量,
  count(case when esc.id = 3 then 1 else null end) as  少儿展示量,
  count(1) as 总展示量
from el_sale_manyservice_log esml left join el_sale_category esc on esc.id = esml.cateid
where from_unixtime(esml.addtime,'%Y-%m-%d') between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
group by from_unixtime(esml.addtime,'%Y-%m-%d') #with rollup
)a
left join (
select 
  from_unixtime(eso.create_time,'%Y-%m-%d') as order_date,
  count(eso.id) as 课程包订单数,
  sum(totalprice) as 课程包金额,
  sum(case when totalprice > 1 then 1 else 0 end) as '课程包订单数(订单金额>1)',  # 订单金额>1
  sum(case when totalprice > 1 then totalprice else 0 end) as '课程包金额(订单金额>1)' # 订单金额>1
from  el_sale_order eso
where from_unixtime(eso.create_time,'%Y-%m-%d') between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
group by from_unixtime(eso.create_time,'%Y-%m-%d') 
) b on a.add_date = b.order_date
left join (
select 
 from_unixtime(a.ctime,'%Y-%m-%d') as order_date,
 sum(case when a.price > 1 then 1 else 0 end) as '静默课程数(订单金额>1)',
 sum(case when a.price > 1 then a.price else 0 end)  as '静默金额(订单金额>1)'
from  el_zy_order_course a left join el_zy_video  b on b.id = a.video_id
where 
 a.channel_sources = 0
and a.pay_status = 3
and FROM_UNIXTIME(a.ctime,'%Y-%m-%d') between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
group by  FROM_UNIXTIME(a.ctime,'%Y-%m-%d')
)c on a.add_date = c.order_date;"""   

sql2 = """
select 
  from_unixtime(esml.addtime,'%Y-%m-%d') as 分配二维码时间,
  esp.uid as 客服id,
  eu.uname as 客服姓名,
  esg.title as  团队,
  esg1.title as  销售组,  
  count(case when esml.cateid = 1 then 1 else null end) as  软笔展示量,
  count(case when esml.cateid = 2 then 1 else null end) as  成人硬笔展示量,
  count(case when esml.cateid = 3 then 1 else null end) as  少儿展示量,
  count(1) as 总展示量
from el_sale_manyservice_log esml
  left join el_sale_video esv on esv.id = esml.baoid
  left join el_sale_manyservice esm on esml.manyid = esm.id
  left join el_sale_people esp on esp.id = esm.saleman_id
  left join el_user eu on eu.uid = esp.uid
  left join el_sale_group esg on esg.sale_group_id = esp.team_id
  left join el_sale_group esg1 on esg1.sale_group_id = replace(esp.group_id,',','')
where
   from_unixtime(esml.addtime,'%Y-%m-%d') between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
group by from_unixtime(esml.addtime,'%Y-%m-%d'),esp.uid,eu.uname,esg.title,esg1.title;
"""

sql3 = """
select 
    from_unixtime(eso.create_time,'%Y-%m-%d') as 成单日期,
    esg.title as 团队,
    esg1.title as 销售组,  
    esp.uid as 销售id,  
    eu.uname as 销售员姓名,  
    sum(case when esv.business_id = 1 then 1 else 0 end) as  软笔订单数,
    sum(case when esv.business_id = 2 then 1 else 0 end) as  硬笔订单数,
    sum(case when esv.business_id = 3 then 1 else 0 end) as  少儿订单数,
    count(eso.sale_video_id) as '总销售订单数(订单金额>1)' ,
    sum(case when esv.business_id = 1 then eso.totalprice else 0 end) as  软笔销售金额,
    sum(case when esv.business_id = 2 then eso.totalprice else 0 end) as  硬笔销售金额,
    sum(case when esv.business_id = 3 then eso.totalprice else 0 end) as  少儿销售金额,
    sum(eso.totalprice) as '总销售金额(订单金额>1)'
from el_sale_order eso left join el_sale_video esv on eso.sale_video_id = esv.id
 left join el_sale_people esp on esp.uid = eso.sale_id
 left join el_user eu on eu.uid = esp.uid
 left join el_sale_group esg on esg.sale_group_id = esp.team_id
 left join el_sale_group esg1 on esg1.sale_group_id = replace(esp.group_id,',','')
where   
    FROM_UNIXTIME(eso.create_time,'%Y-%m-%d')  between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
 and esp.team_id !=0
 and eso.totalprice > 1
group by 1,2,3,4,5
order by 1,2,3,sum(eso.totalprice) desc;
"""

sql4 = """
select 
  from_unixtime(esml.addtime,'%Y-%m-%d') as 分配二维码时间,
  esg.title as 团队,
  esg1.title as  销售组名,  
  count(case when esml.cateid = 1 then 1 else null end) as  软笔展示量,
  count(case when esml.cateid = 2 then 1 else null end) as  成人硬笔展示量,
  count(case when esml.cateid = 3 then 1 else null end) as  少儿展示量,
  count(esp.uid) as 总展示量
from el_sale_manyservice esm 
 left join el_sale_manyservice_log esml on esml.manyid = esm.id
 left join el_sale_video esv on esv.id = esml.baoid
 left join el_sale_people esp on esp.id = esm.saleman_id
 left join el_user eu on eu.uid = esp.uid
 left join el_sale_group esg on esg.sale_group_id = esp.team_id
 left join el_sale_group esg1 on esg1.sale_group_id = replace(esp.group_id,',','')
where
   from_unixtime(esml.addtime,'%Y-%m-%d') between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
group by from_unixtime(esml.addtime,'%Y-%m-%d'),esg.title,esg1.title;
"""

sql5 = """
select 
    from_unixtime(eso.create_time,'%Y-%m-%d') as 成单日期,
    esg.title as 团队,
    esg1.title as 销售组,  
    sum(case when esv.business_id = 1 then 1 else 0 end) as  软笔订单数,
    sum(case when esv.business_id = 2 then 1 else 0 end) as  硬笔订单数,
    sum(case when esv.business_id = 3 then 1 else 0 end) as  少儿订单数,
    count(eso.sale_video_id) as '销售订单数(订单金额>1)',
    sum(case when esv.business_id = 1 then eso.totalprice else 0 end) as  软笔销售金额,
    sum(case when esv.business_id = 2 then eso.totalprice else 0 end) as  硬笔销售金额,
    sum(case when esv.business_id = 3 then eso.totalprice else 0 end) as  少儿销售金额,
    sum(eso.totalprice) as '总销售金额(订单金额>1)'
from el_sale_order eso left join el_sale_video esv on eso.sale_video_id = esv.id
 left join el_zy_video ezv on ezv.id = esv.v_id
 left join el_sale_people esp on esp.uid = eso.sale_id
 left join el_user eu on eu.uid = esp.uid
 left join el_sale_group esg on esg.sale_group_id = esp.team_id
 left join el_sale_group esg1 on esg1.sale_group_id = replace(esp.group_id,',','')
where   
    FROM_UNIXTIME(eso.create_time,'%Y-%m-%d')  between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
 and esp.team_id !=0
 and eso.totalprice > 1
group by 1,2,3
order by 1,2,3,sum(eso.totalprice) desc;
"""

sql6 = """
select 
 from_unixtime(eso.create_time,'%Y-%m-%d') as 成单时间,
 eso.sale_video_id as 课程包id,
 esv.name as 销售课程包, 
 eso.buy_id as 用户id,
 from_unixtime(eu1.ctime,'%Y-%m-%d') as 用户注册时间,
 eso.totalprice as 订单金额,
 eso.sale_id as 销售人员id,
 eu.uname as 销售人员,
 esg.title as  团队,
 esg1.title as 销售组
from el_sale_order eso left join el_sale_video esv on eso.sale_video_id = esv.id
left join el_sale_people esp on esp.uid = eso.sale_id
left join el_user eu on eu.uid = esp.uid
left join el_user eu1 on eu1.uid = eso.buy_id
left join el_sale_group esg on esg.sale_group_id = esp.team_id
left join el_sale_group esg1 on esg1.sale_group_id = replace(esp.group_id,',','')
where   
    FROM_UNIXTIME(eso.create_time,'%Y-%m-%d')  between date_sub(CURDATE(),interval 3 day) and date_sub(CURDATE(),interval 1 day)
 and esp.team_id !=0
and eso.totalprice >1 ;
"""

In [10]:
def main(path):
    sqls = {"销售总体单日汇总":sql1,"销售个人展示汇总":sql2,"销售个人业绩汇总":sql3,
       "销售组展示汇总":sql4,"销售组业绩汇总":sql5,"销售订单明细":sql6,}
    for sql,name in zip(sqls.values(),sqls.keys()):
        a,b = get_file(sql,name,path)
    send(path)

In [11]:
path = "email_data"
main(path)

2020-07-22 16:16:40.632813
email_data/销售总体单日汇总2020-07-21.xlsx
2020-07-22 16:16:47.334928
email_data/销售个人展示汇总2020-07-21.xlsx
2020-07-22 16:16:48.442945
email_data/销售个人业绩汇总2020-07-21.xlsx
2020-07-22 16:16:49.573280
email_data/销售组展示汇总2020-07-21.xlsx
2020-07-22 16:16:50.596823
email_data/销售组业绩汇总2020-07-21.xlsx
2020-07-22 16:16:51.715094
email_data/销售订单明细2020-07-21.xlsx


send: 'ehlo [192.168.60.59]\r\n'
reply: b'250-smtp.qq.com\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-SIZE 73400320\r\n'
reply: b'250-AUTH LOGIN PLAIN\r\n'
reply: b'250-AUTH=LOGIN\r\n'
reply: b'250-MAILCOMPRESS\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'smtp.qq.com\nPIPELINING\nSIZE 73400320\nAUTH LOGIN PLAIN\nAUTH=LOGIN\nMAILCOMPRESS\n8BITMIME'
send: 'AUTH PLAIN AGxvdWp1bmppZUBscHQub25leG1haWwuY29tAFhDOG9VMk1wMjhrZjYyODY=\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<loujunjie@lpt.onexmail.com> size=61926\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<zhangyuexia@liupinedu.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<zhangyuexia@liupinedu.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>

邮件发送成功
2020-07-22 16:16:55.308578


reply: b'250 Ok: queued as \r\n'
reply: retcode (250); Msg: b'Ok: queued as'
data: (250, b'Ok: queued as')
send: 'quit\r\n'
reply: b'221 Bye\r\n'
reply: retcode (221); Msg: b'Bye'
